In [1]:
import os 
import sys 
import json 
import logging 
from pathlib import Path 
from functools import cache
from itertools import product

# Required when developing in a jupyter-notebook environment 
cur_path = os.path.abspath("../..")
if cur_path not in sys.path: 
    sys.path.append(cur_path)

import numpy as np 
import pandas as pd 
import altair as alt 
from altair import datum
from dotenv import load_dotenv
from subgrounds.subgrounds import Subgrounds, Subgraph
from subgrounds.subgraph import SyntheticField
from subgrounds.pagination import ShallowStrategy

# Required when developing in a jupyter-notebook environment 
assert load_dotenv('../../../.env')

from utils_notebook.utils import ddf, remove_prefix, load_subgraph, remove_keys
from utils_notebook.vega import condition_union, output_chart, apply_css 
from utils_notebook.testing import validate_season_series
from utils_notebook.constants import ADDR_BEANSTALK

In [2]:
sg, bs = load_subgraph()

In [3]:
@cache
def query_rewards(refresh=None): 
    bs.Reward.fertilized_beans_daily = SyntheticField(
        lambda toFertilizer: float(toFertilizer) / 1e6, 
        SyntheticField.FLOAT, 
        bs.Reward.toFertilizer
    ) 
    q = bs.Query.rewards(orderBy="blockNumber", orderDirection="asc", first=10000)
    df = sg.query_df(
        [
            q.season, 
            q.fertilized_beans_daily, 
        ], 
        pagination_strategy=ShallowStrategy
    )
    return remove_prefix(df, 'rewards_').sort_values('season')

@cache 
def query_fertilizer_tokens(refresh=None): 
    bs.FertilizerToken.fert = bs.FertilizerToken.supply 
    bs.FertilizerToken.start_bpf = bs.FertilizerToken.startBpf / 1e6 
    bs.FertilizerToken.end_bpf = SyntheticField(
      lambda _id: float(_id) / 1e6, 
      SyntheticField.FLOAT,
      bs.FertilizerToken.id, 
    )
    ft = bs.Query.fertilizerTokens(
        first=10000, 
        orderBy="humidity", 
        orderDirection="desc"
    )
    df = sg.query_df(
        [
            ft.season, 
            ft.fert, 
            ft.start_bpf, 
            ft.end_bpf, 
        ],
        pagination_strategy=ShallowStrategy
    )
    return remove_prefix(df, "fertilizerTokens_")

In [4]:
# fertilizer emissions (incomplete season axis, all seasons unique)
df_rewards = query_rewards(refresh=5).copy()
df_rewards.head()
# df_rewards = df_rewards[['season', 'fertilized_beans_daily']]
# validate_season_series(df_rewards, allow_missing=True)
# df_rewards.head()

,season,fertilized_beans_daily
0,6076,2266.788451
1,6077,4557.830445
2,6078,6936.066574
3,6079,9357.109094
4,6080,13383.907966


In [5]:
df_fert = query_fertilizer_tokens(refresh=5).copy()
df_fert = df_fert.merge(df_rewards, how="outer", on="season").sort_values('season')
df_fert['fertilized_beans_daily'] = df_fert.fertilized_beans_daily.fillna(0)
df_fert['fertilized_beans_cumulative'] = df_fert.fertilized_beans_daily.cumsum()
df_fert['unfertilized_beans_cumulative'] = ((df_fert.fert * df_fert.end_bpf).cumsum() - df_fert.fertilized_beans_cumulative).ffill()
df_fert = df_fert[
    ['season', 'fertilized_beans_cumulative', 'unfertilized_beans_cumulative']
].groupby("season").agg({
    "fertilized_beans_cumulative": "max", 
    "unfertilized_beans_cumulative": "max"
}).reset_index()
validate_season_series(df_fert, allow_missing=True)
df_fert.head()

,season,fertilized_beans_cumulative,unfertilized_beans_cumulative
0,6074,0.000000,8.643268e+07
1,6075,0.000000,8.644328e+07
2,6076,2266.788451,8.654976e+07
3,6077,6824.618896,8.658356e+07
4,6078,13760.685470,8.659188e+07


In [6]:
@cache 
def query_field(refresh=None) -> pd.DataFrame: 
    field_snaps = bs.Query.fieldDailySnapshots(
        orderBy="season", 
        orderDirection="asc", 
        first=10000, 
        where={"field": ADDR_BEANSTALK}
    )
    df_field = sg.query_df(
        [
            field_snaps.season, 
            field_snaps.newHarvestedPods, 
            field_snaps.newHarvestablePods, 
            field_snaps.podIndex, 
        ], 
        pagination_strategy=ShallowStrategy
    )
    return df_field 

In [7]:
df_field = query_field(refresh=3).copy()
df_field = remove_prefix(df_field, "fieldDailySnapshots_")
df_field = df_field.sort_values("season")
df_field['pods_harvestable_daily'] = (df_field.newHarvestablePods / 10**6)
df_field['pods_harvested_daily'] = df_field.newHarvestedPods / 10**6
df_field = df_field.drop(columns=['newHarvestablePods', 'newHarvestedPods'])
df_field = df_field.groupby('season').agg({
    # handles edge case for season 6074 which occurred multiple times 
    "pods_harvestable_daily": "sum", 
    "pods_harvested_daily": "sum", 
    "podIndex": "max"
}).reset_index()
df_field['pods_issued_cumulative'] = df_field.podIndex / 10**6
df_field['pods_issued_daily'] = df_field.pods_issued_cumulative - df_field.pods_issued_cumulative.shift(1).fillna(0)
df_field['pods_harvestable_cumulative'] = df_field.pods_harvested_daily.cumsum() # TODO: factor in harvestable daily 
df_field = df_field.drop(columns=['podIndex'])
validate_season_series(df_field, allow_missing=True)
df_field.tail()

,season,pods_harvestable_daily,pods_harvested_daily,pods_issued_cumulative,pods_issued_daily,pods_harvestable_cumulative
305,7305,0.0,0.0,8.286361e+08,4117.776547,5.756021e+07
306,7329,0.0,0.0,8.286403e+08,4198.016655,5.756021e+07
307,7353,0.0,0.0,8.286403e+08,0.019026,5.756021e+07
308,7377,0.0,0.0,8.286403e+08,0.027175,5.756021e+07
309,7398,0.0,0.0,8.286403e+08,0.089900,5.756021e+07


In [8]:
def silo_emissions_pre_replant() -> pd.DataFrame: 
    """Temporary solution to subgraph not having silo emissions pre-replant 
    
    Data was downloaded from dune 
    """
    with Path("data/SupplyIncrease.json").open('r') as f: 
        data = json.loads(f.read())
    data = [remove_keys(d['data'], ['__typename']) for d in data]
    df_supply_inc = pd.DataFrame(data)[['season', 'newSilo']]
    return df_supply_inc

@cache 
def query_silo(refresh=None) -> pd.DataFrame: 
    silo_snaps = bs.Query.siloDailySnapshots(
        orderBy="season", 
        orderDirection="asc", 
        first=10000, 
        where={"silo": ADDR_BEANSTALK}
    )
    df = sg.query_df(
        [
            silo_snaps.season, 
            silo_snaps.dailyBeanMints, 
            # silo_snaps.totalBeanMints, # add back when subgraph includes historical data 
        ], 
        pagination_strategy=ShallowStrategy
    )
    return df 

In [9]:
# process post-replant silo data (subgraph)
df_silo = query_silo(refresh=1).copy()
df_silo = remove_prefix(df_silo, "siloDailySnapshots_")
df_silo = df_silo.loc[df_silo.season != 5903] # Subgraph bug probably? 
assert df_silo.season.min() == 6074, "If this fails, then subgraph was fixed to include historical data."
df_silo = df_silo.rename(columns={"dailyBeanMints": "silo_emissions_daily"})
# process pre-replant silo data (downloaded from dune)
df_silo_old = silo_emissions_pre_replant()
df_silo_old = df_silo_old.rename(columns={"newSilo": "silo_emissions_daily"})
# Combine pre and post replant data (no seasons in common so outer join)
df_silo = df_silo.merge(df_silo_old, how="outer")
assert set(df_silo.columns) == set(['season', 'silo_emissions_daily'])
df_silo = df_silo.sort_values("season")
df_silo = df_silo.groupby('season').agg({
    # handles edge case for season 6074 which occurred multiple times 
    "silo_emissions_daily": "sum", 
}).reset_index()
df_silo['silo_emissions_daily'] /= 10**6
df_silo['silo_emissions_cumulative'] = df_silo.silo_emissions_daily.cumsum()
validate_season_series(df_silo, allow_missing=True)
df_silo.head() 

,season,silo_emissions_daily,silo_emissions_cumulative
0,3,31.650670,31.650670
1,4,17.760260,49.410930
2,5,43.709604,93.120534
3,18,0.342173,93.462707
4,21,676.195254,769.657961


In [10]:
@cache 
def query_seasons(refresh=None) -> pd.DataFrame: 
    seasons = bs.Query.seasons(
        first=10000, orderBy="season", orderDirection="asc"
    )
    bs.Season.bean_supply = bs.Season.beans / 1e6
    df = sg.query_df([
        seasons.season, 
        seasons.timestamp, 
        seasons.bean_supply, 
    ], pagination_strategy=ShallowStrategy)
    df = remove_prefix(df, 'seasons_')
    return df 

In [11]:
df_szns = query_seasons(refresh=1)
df_szns['timestamp'] = pd.to_datetime(df_szns.timestamp, unit='s')
validate_season_series(df_szns, allow_missing=False)
df_szns = df_szns.loc[df_szns.season >= 2] # timestamps are wrong for season 0 and 1 
df_szns.head()

,season,timestamp,bean_supply
2,2,2021-08-07 00:06:08,2078.821989
3,3,2021-08-07 01:07:38,4089.294648
4,4,2021-08-07 02:09:28,6085.876897
5,5,2021-08-07 03:07:35,8108.408490
6,6,2021-08-07 04:11:23,10087.230479


In [12]:
# data pre-processing 
df = df_szns.merge(
    df_fert, how='left', on='season'
).merge(
    df_field, how='left', on='season'
).merge(
    df_silo, how='left', on='season'
)
assert len(df) == len(df_szns)
df = df.ffill().fillna(0) # Not technically correct but close enough 
df['total_debt'] = (
    df.pods_issued_cumulative
    + df.unfertilized_beans_cumulative
) 
df['total_credit'] = (
    df.fertilized_beans_cumulative
    + df.silo_emissions_cumulative 
    + df.pods_harvestable_cumulative
)
df['debt_credit_ratio'] = df.total_debt / df.total_credit
df['fertilizer_adjusted_pod_rate'] = df.total_debt / df.bean_supply 
metrics_credit = [
    'silo_emissions_cumulative',
    'pods_harvestable_cumulative',
    'fertilized_beans_cumulative', 
]
metrics_debt = [
    'unfertilized_beans_cumulative', 
    'pods_issued_cumulative', 
]
metrics_credit_debt_aggregate = [
    'total_debt', 
    'total_credit', 
]
metrics_meta = [
    'debt_credit_ratio', 
    'fertilizer_adjusted_pod_rate', 
]
metrics = metrics_credit + metrics_debt + metrics_credit_debt_aggregate + metrics_meta
columns = ['timestamp'] + metrics 
df = df[columns]
df = df.resample("W", on="timestamp").last().drop(columns="timestamp").reset_index()
df_mask = df.silo_emissions_cumulative.isna()
timestamp_min = df.timestamp.values[0]
timestamp_exploit = df[df_mask].timestamp.values[0]
timestamp_replant = df[df_mask].timestamp.values[-1]
df = df.dropna()
source = df.melt(
    id_vars=['timestamp'], 
    value_vars=metrics, 
).sort_values(["timestamp", "variable"]).reset_index(drop=True)
print(len(source))
source.head(10)

414


,timestamp,variable,value
0,2021-08-08,debt_credit_ratio,0.500192
1,2021-08-08,fertilized_beans_cumulative,0.000000
2,2021-08-08,fertilizer_adjusted_pod_rate,0.155825
3,2021-08-08,pods_harvestable_cumulative,15526.430505
4,2021-08-08,pods_issued_cumulative,18366.370137
5,2021-08-08,silo_emissions_cumulative,21192.194431
6,2021-08-08,total_credit,36718.624936
7,2021-08-08,total_debt,18366.370137
8,2021-08-08,unfertilized_beans_cumulative,0.000000
9,2021-08-15,debt_credit_ratio,0.248030


In [27]:
# alt.data_transformers.disable_max_rows()

dropdown = alt.binding_select(options=['ym', 'ymd'], name='Aggregation Level')
selection = alt.selection_single(name="agglevel", fields=['AggLevel'], bind=dropdown, init={"AggLevel": 'ymd'})
selection_rule = alt.selection_single(
    fields=['tstamp'], nearest=True, on='mouseover', empty='none', clear='mouseout'
)
colors = {
    # credit components 
    'fertilized_beans_cumulative': '#57cc99', # green   
    'pods_harvestable_cumulative': '#38a3a5', # mid blue 
    'silo_emissions_cumulative': '#22577a', # navy blue 
    # debt components 
    'unfertilized_beans_cumulative': "#ef9b20", # Magenta 50
    'pods_issued_cumulative': '#fa4d56', # Red 50
    # overall 
    'total_debt': '#9f1853', # Magenta 70 
    'total_credit': '#80ed99', # mint green 
    'debt_credit_ratio': '#ffc300', # gold 
    'fertilizer_adjusted_pod_rate': '#5e60ce' # purple-ish
}
format_decimal = ",d"
format_percent = ".2%"
tooltip_formats = {
    'fertilized_beans_cumulative': format_decimal,
    'unfertilized_beans_cumulative': format_decimal,
    'pods_harvestable_cumulative': format_decimal,
    'silo_emissions_cumulative': format_decimal,
    'pods_issued_cumulative': format_decimal,
    'total_debt': format_decimal,
    'total_credit': format_decimal,
    'debt_credit_ratio': format_percent, 
    'fertilizer_adjusted_pod_rate': format_percent,
}
assert set(colors.keys()) == set(metrics)
assert set(tooltip_formats.keys()) == set(metrics)

# ngl I popped off on this one 
stack_order_expr = (
    # creates numeric stack order key encoding both x position and order of stacked area labels into single value 
    ' '.join(
        [
            f"datum.variable === '{m}' ? {i} : " 
            for i, m in enumerate(reversed(metrics))
        ]
    ) 
    + str(len(metrics))
)
stack_order_expr = f'time(datum.tstamp) + ({stack_order_expr})'

base = (
    alt.Chart(source)
    .properties(height=225, width=500)
    .transform_timeunit(
        ymd="yearmonthdate(timestamp)", 
        ym="yearmonth(timestamp)", 
    )
    .transform_calculate(
        tstamp="datum[agglevel.AggLevel]", 
    )
    .transform_aggregate(
        groupby=["tstamp", 'variable'], rvalue='max(value)'
    )
    .transform_calculate(stack_order=stack_order_expr)
    .encode(
        x=alt.X(
            "tstamp:O", 
            axis=alt.Axis(
                formatType="time", 
                ticks=False, 
                labelExpr="timeFormat(toDate(datum.value), '%b %Y')", 
                labelOverlap=True, 
                labelSeparation=25, 
                labelPadding=5, 
                title="Date", 
                labelAngle=0, 
            ), 
        ),   
    )
)
base_bdv = (
    base
    .encode(
        y=alt.Y("rvalue:Q", axis=alt.Axis(title="BDV")),
        color=alt.Color(
            "variable:N", 
            scale=alt.Scale(
                domain=metrics_credit + metrics_debt + metrics_credit_debt_aggregate, 
                range=[colors[m] for m in metrics_credit + metrics_debt + metrics_credit_debt_aggregate]
            ),
            legend=alt.Legend(title="Legend")
        ),
        order=alt.Order('stack_order:Q', sort='ascending')
    )
)
base_ratio = base.encode(
    y=alt.Y("rvalue:Q", axis=alt.Axis(title="Percent", format=",%")),
    color=alt.Color(
        "variable:N", 
        scale=alt.Scale(
            domain=metrics_meta, range=[colors[m] for m in metrics_meta]
        ),
        legend=alt.Legend(title="Legend")
    ),
)
rule_exploit = (
    # selection captures nearest timestamp (for current mouse position) 
    # tooltip rendered uses this data point (pivoted, so we have all data for this timestamp) 
    base
    .transform_pivot('variable', value='rvalue', groupby=['tstamp'])
    .transform_filter("year(datum.tstamp) === 2022 && month(datum.tstamp) === 3 && date(datum.tstamp) === 17")
    .mark_rule(opacity=1, color='#474440', strokeDash=[2.5,1])
)
rule = (
    # selection captures nearest timestamp (for current mouse position) 
    # tooltip rendered uses this data point (pivoted, so we have all data for this timestamp) 
    base
    .transform_pivot('variable', value='rvalue', groupby=['tstamp'])
    .mark_rule(opacity=0)
    .encode(
        tooltip=(
            [alt.Tooltip('tstamp:O', timeUnit="yearmonthdate", title="date")] + 
            [
                alt.Tooltip(
                    f'{m}:Q', 
                    format=tooltip_formats[m], 
                    title=m.replace("_", " ").replace(" cumulative", "")
                ) for m in metrics
            ] 
        ) 
    )
    .add_selection(selection_rule)
)
credit = (
    base_bdv
    .mark_bar()
    .transform_filter(condition_union("==", "|", metrics_credit))
)
debt = (
    base_bdv
    .mark_bar()
    .transform_filter(condition_union("==", "|", metrics_debt))
) 
lines_debt_credit = (
    base_bdv
    .mark_line()
    .transform_filter(condition_union('==', '|', metrics_credit_debt_aggregate))
) 
line_ratio = (
    base_ratio
    .mark_line()
    .transform_filter(condition_union('==', '|', metrics_meta))
)
point_ratio = (
    base_ratio
    .mark_point(size=7)
    .transform_filter(condition_union('==', '|', metrics_meta))
)

c = (
    alt
    .vconcat(
        alt.layer(debt, credit, lines_debt_credit, rule, rule_exploit).properties(title="Beanstalk Credit Profile"),
        alt.layer(line_ratio, point_ratio, rule, rule_exploit).properties(title="Beanstalk Credit Metrics"),
    )
    .resolve_legend(color="independent")
    .resolve_axis(y="independent")
    .resolve_scale(y="independent", color="independent")
    .add_selection(selection) 
)


css_lines = [
    "div.chart-wrapper { display: flex; flex-direction: column; }", 
    "form.vega-bindings { display: block; order: -1; }", 
    "canvas { order: 1 }", 
    "#vg-tooltip-element tr:nth-child(1) { font-weight: bold }", 
    "#vg-tooltip-element tr:nth-child(1) td:first-child { opacity: 0 }\n", 
]
for i, m in enumerate(metrics): 
    # i+2 bc css uses one based indexing and we also don't color timestamp (first element in tooltip)
    css_lines.append(
        "#vg-tooltip-element tr:nth-child(%d) td:first-child { color: %s }\n" % (i+2, colors[m])
    )
css = "\n".join(css_lines)
    
apply_css(css) 
c

alt.VConcatChart(...)

In [19]:
output_chart(c, css=css)

<IPython.core.display.JSON object>